In [27]:
import numpy as np
import sklearn
import pandas as pd
MergeListe = []
#Noch offene Todos
    #Fehlerwerte definieren
    #variable als input string?
    #nochmal überprüfen, ob die richtigen Tabellen benutzt wurden
    #überprüfen ob daten bei regen stimmen können, da sehr oft 0.0
    #Regressionsanalyse? Sinnvoll?

In [28]:
#Read air temperature data; Temperature in °C and rename column 
#only keep relevant time frame
#put pd into a Merge List

# ns all datasets need to be stored in same repository as this notebook

Temp_Dataset = pd.read_csv("Temperatur_tu_stunde_19810101_20181231_01420.txt", 
                header = 0,
                index_col="DateTime",
                names = ["MeasuringStation",
                        "DateTime",
                        "QN_9",
                        "Temperature",
                        "RelativeAirHumidity",
                        "eor"], 
                sep=";",
                decimal=".",
                usecols=["DateTime","Temperature","RelativeAirHumidity"])
Temp_Dataset = Temp_Dataset[Temp_Dataset.index>= 2015050100]
Temp_Dataset = Temp_Dataset[Temp_Dataset.index <2017010100]
print(Temp_Dataset)
MergeListe.append(Temp_Dataset)

            Temperature  RelativeAirHumidity
DateTime                                    
2015050100          8.0                 95.0
2015050101          7.8                 95.0
2015050102          7.7                 97.0
2015050103          7.2                 97.0
2015050104          6.0                 98.0
...                 ...                  ...
2016123119         -3.6                 99.0
2016123120         -3.5                 99.0
2016123121         -3.7                 98.0
2016123122         -3.7                 99.0
2016123123         -3.9                 99.0

[14664 rows x 2 columns]


In [29]:
#Notiz: ich würde eigentlich gerne alles mit Schleifen machen lassen. 
#Also alle wichtigen Dateien in einem Ordner haben und das ganze dann durchiterieren. 
#Je nach Zeitaufwand und Lust könnte ich es ja mal versuchen 

In [30]:
#Read Data for SunshineDuration
#in min per hour 
#Same procedure as before 

Sun_Dataset = pd.read_csv("sonne_sd_stunde_19510101_20181231_01420.txt", 
                header = 0,
                index_col="DateTime",
                names = ["MeasuringStation",
                        "DateTime",
                        "QN_7",
                        "SunshineDuration",
                        "eor"], 
                sep=";",
                decimal=".",
                usecols=["DateTime","SunshineDuration"])
Sun_Dataset = Sun_Dataset[Sun_Dataset.index>= 2015050100]
Sun_Dataset = Sun_Dataset[Sun_Dataset.index <2017010100]
print(Sun_Dataset)
MergeListe.append(Sun_Dataset)

            SunshineDuration
DateTime                    
2015050103               0.0
2015050104               0.0
2015050105               0.0
2015050106               0.0
2015050107               1.0
...                      ...
2016123116               0.0
2016123117               0.0
2016123118               0.0
2016123119               0.0
2016123120               0.0

[10998 rows x 1 columns]


In [31]:
#Read cloudiness Data 
#Cloudiness ranked from 0 to 8; -1 for the case, that sky could not bee seen
#Same procedure as before

Cloud_Dataset = pd.read_csv("Cloudiness_n_stunde_19490101_20181231_01420.txt", 
                header = 0,
                index_col="DateTime",
                names = ["MeasuringStation",
                        "DateTime",
                        "QN_8",
                        "IndexHowMeasurementIsTaken",
                        "TotalCloudCover",
                        "eor"], 
                sep=";",
                decimal=".",
                usecols=["DateTime","TotalCloudCover"])
Cloud_Dataset = Cloud_Dataset[Cloud_Dataset.index>= 2015050100]
Cloud_Dataset = Cloud_Dataset[Cloud_Dataset.index <2017010100]
print(Cloud_Dataset)
MergeListe.append(Cloud_Dataset)

            TotalCloudCover
DateTime                   
2015050100                7
2015050101                8
2015050102                7
2015050103                6
2015050104                6
...                     ...
2016123119                8
2016123120                8
2016123121                8
2016123122                8
2016123123                8

[14664 rows x 1 columns]


In [32]:
#Notiz: Es gibt noch Infos über die Niederschlagsform, allerdngs relativ komplex. Wollen wir diese mit reinnehmen oder nicht?
# Hier sind ist die Form erstmal außen vor gelassen

In [33]:
#Read Data for Precipitation
#in mm per hour 
#Same procedure as before 
Precipitation_Dataset = pd.read_csv("Niederschlag_rr_stunde_19950901_20181231_01420.txt", 
                header = 0,
                index_col="DateTime",
                names = ["MeasuringStation",
                        "DateTime",
                        "QN_8",
                        "PrecipitationHeight",
                        "PrecipitationBool",
                        "FormOfPrecipitation",
                        "eor"], 
                sep=";",
                decimal=".",
                usecols=["DateTime","PrecipitationHeight"])
Precipitation_Dataset = Precipitation_Dataset[Precipitation_Dataset.index>= 2015050100]
Precipitation_Dataset = Precipitation_Dataset[Precipitation_Dataset.index <2017010100]
print(Precipitation_Dataset)
MergeListe.append(Precipitation_Dataset)

            PrecipitationHeight
DateTime                       
2015050100                  0.0
2015050101                  0.0
2015050102                  0.0
2015050103                  0.0
2015050104                  0.0
...                         ...
2016123119                  0.0
2016123120                  0.0
2016123121                  0.0
2016123122                  0.0
2016123123                  0.0

[14656 rows x 1 columns]


In [34]:
#Reading visibility Data
# in meters
# same as before
Visibility_Dataset = pd.read_csv("Sicht_vv_stunde_19490101_20181231_01420.txt", 
                header = 0,
                index_col="DateTime",
                names = ["MeasuringStation",
                        "DateTime",
                        "QN_8",
                        "IndexHowMeasurementIsTaken",
                        "Visibility",
                        "eor"], 
                sep=";",
                decimal=".",
                usecols=["DateTime","Visibility"])
Visibility_Dataset = Visibility_Dataset[Visibility_Dataset.index>= 2015050100]
Visibility_Dataset = Visibility_Dataset[Visibility_Dataset.index <2017010100]
print(Visibility_Dataset)
MergeListe.append(Visibility_Dataset)

            Visibility
DateTime              
2015050100       30000
2015050101       30000
2015050102       35000
2015050103       35000
2015050104       30000
...                ...
2016123119        2600
2016123120        1900
2016123121        1900
2016123122        1900
2016123123        1900

[14664 rows x 1 columns]


In [35]:
#Read wind Data
#In m/s
#same procedure as before 
Wind_Dataset = pd.read_csv("Wind_ff_stunde_19670101_20181231_01420.txt", 
                header = 0,
                index_col="DateTime",
                names = ["MeasuringStation",
                        "DateTime",
                        "QN_3",
                        "MeanWindSpeed",
                        "MeanWindDirection",
                        "eor"], 
                sep=";",
                decimal=".",
                usecols=["DateTime","MeanWindSpeed"])
Wind_Dataset = Wind_Dataset[Wind_Dataset.index>= 2015050100]
Wind_Dataset = Wind_Dataset[Wind_Dataset.index <2017010100]
print(Wind_Dataset)
MergeListe.append(Wind_Dataset)

            MeanWindSpeed
DateTime                 
2015050100            2.2
2015050101            2.2
2015050102            1.5
2015050103            1.5
2015050104            0.6
...                   ...
2016123119            1.9
2016123120            1.7
2016123121            1.7
2016123122            2.1
2016123123            1.5

[14664 rows x 1 columns]


In [36]:
#joining all existing data frames to one by using the index 
#-> problem: the dataframes are stored as List elements
#Wetterdaten as panda
#for Datensatz in MergeListe:
#    Wetterdaten = Wetterdaten.merge(Datensatz, left_index=True, right_index=True)
#print(Wetterdaten)  
#Wetterdaten = MergeListe
#Wetterdaten[0].merge(Wetterdaten[1:])

# Merge two or more Dataframes on index of both the dataframes
Wetterdaten = pd.concat(MergeListe, axis=1)
print(Wetterdaten)  
Wetterdaten.describe()

            Temperature  RelativeAirHumidity  SunshineDuration  \
DateTime                                                         
2015050100          8.0                 95.0               NaN   
2015050101          7.8                 95.0               NaN   
2015050102          7.7                 97.0               NaN   
2015050103          7.2                 97.0               0.0   
2015050104          6.0                 98.0               0.0   
...                 ...                  ...               ...   
2016123119         -3.6                 99.0               0.0   
2016123120         -3.5                 99.0               0.0   
2016123121         -3.7                 98.0               NaN   
2016123122         -3.7                 99.0               NaN   
2016123123         -3.9                 99.0               NaN   

            TotalCloudCover  PrecipitationHeight  Visibility  MeanWindSpeed  
DateTime                                                       

,Temperature,RelativeAirHumidity,SunshineDuration,TotalCloudCover,PrecipitationHeight,Visibility,MeanWindSpeed
count,14664.000000,14664.000000,10998.000000,14664.000000,14656.000000,14664.000000,14664.000000
mean,12.487786,74.610406,15.140389,5.371522,-0.684498,34807.950764,3.336675
std,7.798837,19.621688,22.833367,2.608460,27.365011,18527.366064,2.049207
min,-7.200000,16.000000,0.000000,-1.000000,-999.000000,100.000000,0.000000
25%,6.400000,61.000000,0.000000,3.000000,0.000000,18000.000000,1.800000
50%,12.200000,78.000000,0.000000,7.000000,0.000000,35000.000000,2.900000
75%,17.900000,92.000000,29.000000,7.000000,0.000000,50000.000000,4.400000
max,38.100000,100.000000,60.000000,8.000000,19.100000,81900.000000,14.600000


In [37]:
"""ns handling error values

Assigning numerical error values to 'nan' so that statistical methods are not biased"""

#ns Temperature Error values are -999
#ns relative Air humidity error values are -999
#ns sunshine duration error values are -999
#ns totalcloudcover error values are -999
#ns precipitationheight error values are -999
#ns visibility error values are -999
#ns meanwindspeed error values are not existent

#ns therefore all -999 in the dataframe are replaced by nan
#ns only PrecipitationHeight has error values
#ns create new dataframe to keep original active and have the new one anyway

Wetterdaten_nand = Wetterdaten.replace(-999,np.NaN)

#ns option to overwrite dataframe or not
# Wetterdaten = Wetterdaten_nand


In [38]:
Wetterdaten_nand.describe()

,Temperature,RelativeAirHumidity,SunshineDuration,TotalCloudCover,PrecipitationHeight,Visibility,MeanWindSpeed
count,14664.000000,14664.000000,10998.000000,14664.000000,14645.000000,14664.000000,14664.000000
mean,12.487786,74.610406,15.140389,5.371522,0.065347,34807.950764,3.336675
std,7.798837,19.621688,22.833367,2.608460,0.460074,18527.366064,2.049207
min,-7.200000,16.000000,0.000000,-1.000000,0.000000,100.000000,0.000000
25%,6.400000,61.000000,0.000000,3.000000,0.000000,18000.000000,1.800000
50%,12.200000,78.000000,0.000000,7.000000,0.000000,35000.000000,2.900000
75%,17.900000,92.000000,29.000000,7.000000,0.000000,50000.000000,4.400000
max,38.100000,100.000000,60.000000,8.000000,19.100000,81900.000000,14.600000


In [39]:
Wetterdaten.describe()

,Temperature,RelativeAirHumidity,SunshineDuration,TotalCloudCover,PrecipitationHeight,Visibility,MeanWindSpeed
count,14664.000000,14664.000000,10998.000000,14664.000000,14656.000000,14664.000000,14664.000000
mean,12.487786,74.610406,15.140389,5.371522,-0.684498,34807.950764,3.336675
std,7.798837,19.621688,22.833367,2.608460,27.365011,18527.366064,2.049207
min,-7.200000,16.000000,0.000000,-1.000000,-999.000000,100.000000,0.000000
25%,6.400000,61.000000,0.000000,3.000000,0.000000,18000.000000,1.800000
50%,12.200000,78.000000,0.000000,7.000000,0.000000,35000.000000,2.900000
75%,17.900000,92.000000,29.000000,7.000000,0.000000,50000.000000,4.400000
max,38.100000,100.000000,60.000000,8.000000,19.100000,81900.000000,14.600000
